In [189]:
#get library
import numpy as np
import pandas as pd

In [190]:
test_set = pd.read_csv("testing_set.csv")

In [191]:
#get user_id from testing set
temp = test_set.columns.values[1:]
test_user_id = []
for i in range(len(temp)):
    test_user_id.append(temp[i])
for i in range(len(test_user_id)):
    test_user_id[i] = int(test_user_id[i])
print("ID of users in the testing set is:")
print(test_user_id)

ID of users in the testing set is:
[37015, 37020, 37032, 37034, 37040, 37049, 37052, 37074, 37097, 37100, 37107, 37110, 37117, 37133, 37149, 37163, 37196, 37199, 37219, 37229, 37233, 37236, 37237, 37264, 37267, 37270, 37326, 37327, 37345, 37372, 37383, 37386, 37399, 37405, 37411, 37423, 37425, 37454, 37459, 37464, 37490, 37494, 37502, 37508, 37520, 37524, 37549, 37558, 37576, 37599, 37609, 37621, 37641, 37662, 37674, 37698, 37702, 37720, 37725, 37754, 37781, 37800, 37803, 37806, 37808, 37823, 37873, 37874, 37903, 37916, 37922, 37928, 37965, 37971, 37979, 37985, 37998, 38004, 38009, 38042, 38045, 38079, 38083, 38087, 38100, 38120, 38137, 38152, 38162, 38213, 38233, 38237, 38253, 38263, 38275, 38278, 38290, 38312, 38318, 38322, 38324, 38367, 38381, 38383, 38388, 38433, 38436, 38462, 38467, 38471, 38487, 38494, 38515, 38521, 38525, 38531, 38537, 38541, 38544, 38555, 38557, 38582, 38586, 38599, 38622, 38644, 38732, 38747, 38753, 38757, 38769, 38777, 38790, 38797, 38803, 38813, 38820, 38833

### Get rating of each user

In [192]:
def getRatingVectorOfNthUser(n:int,user_need_to_hide = -1):
    #get the rating vector of n-th user 
    column_n = str(n) 
    rating = test_set[[column_n]].values
    not_minus1_sum,count = 0,0
    for i in rating:
        if i[0] != -1 and i[0] != 0:
            not_minus1_sum += i[0]
            count += 1
    if count != 0:
        mean = round(not_minus1_sum/count,2)
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = mean
    else:
        for i in range(len(rating)):
            if rating[i][0] == -1:
                rating[i][0] = 5
        
    #normalize the rating value
    del not_minus1_sum,count
    #return 70% of rating vector if source is test else 100%
    count = 0
    sum,count = 0,0
    for r in rating:
        if r[0] != 0:
            sum += r[0]
            count += 1
    mean = round(sum/count,2)
    if user_need_to_hide == n:
        select = (count*7)//10 #get 70% of the rating value when we need to hide the choosen user
        temp,index,true_rating = 0,[],[]
        for i in range (len(rating)):
            if rating[i][0] != 0:
                if temp >= select:
                    true_rating.append(rating[i][0])
                    index.append(i)
                    rating[i][0] = 0
                temp += 1
        del temp,select
        return rating,index,true_rating,mean

    return rating

print(getRatingVectorOfNthUser(37020))

[[10.]
 [ 8.]
 [10.]
 ...
 [ 0.]
 [ 0.]
 [ 0.]]


In [193]:
def intoList(vector):
    """
    Parameters : numpy array

    This function takes a numpy array and turn it into a list

    Even though numpy support this by tolist() method but it is not suitable for this case

    Return : a python list

    Time complexity : O(n)
    """

    ans = []
    for i in vector:
        ans.append(i[0])
    return ans
print(intoList(getRatingVectorOfNthUser(37020)))

[10.0, 8.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 7.0, 10.0, 0.0, 10.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [194]:
A = []
chosen_user,idx,true_rating = 37020,None,None
for test_user in test_user_id:
    if test_user != chosen_user:
        added_row = intoList(getRatingVectorOfNthUser(test_user))
    else:
        added_row,idx,true_rating,average = getRatingVectorOfNthUser(test_user,chosen_user)
        added_row = intoList(added_row)
    A.append(added_row)
A = np.array(A)
A = A.T
print(A)


[[ 5. 10.  0. ...  0.  0.  0.]
 [ 0.  8.  0. ...  0.  0.  0.]
 [ 0. 10.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


### DECOMPOSE THE MATRIX INTO 3 SUBMATRIX $U$,$Sigma$,$V^T$
- The current size of the testing matrix is 9724x1000

In [195]:
U,Sigma,VT = np.linalg.svd(A)
print(np.round(U,decimals=2))
print(np.round(Sigma,decimals=1))
print(np.round(VT,decimals=2))

[[-0.05  0.09  0.01 ...  0.    0.    0.  ]
 [-0.02  0.04  0.02 ...  0.    0.    0.  ]
 [-0.04  0.07  0.02 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ...  1.    0.    0.  ]
 [ 0.    0.    0.   ...  0.    1.    0.  ]
 [ 0.    0.    0.   ...  0.    0.    1.  ]]
[1110.4  477.4  383.7  356.5  331.2  320.9  277.7  275.4  249.5  239.7
  228.5  223.4  218.5  212.9  207.7  204.2  199.8  199.2  195.2  192.1
  187.9  187.6  182.8  182.5  180.   178.9  175.   173.6  171.2  169.3
  167.6  165.4  164.9  163.7  162.   160.6  160.   156.4  155.6  155.
  154.6  153.5  152.3  151.7  149.4  149.   148.1  147.2  147.   146.1
  145.3  145.   144.6  143.6  141.7  140.9  139.5  138.5  138.3  137.9
  136.7  135.9  135.5  134.8  134.6  133.9  133.4  133.2  132.3  132.
  130.9  130.4  129.6  129.4  128.6  128.3  127.9  127.4  126.2  126.
  125.5  124.8  124.4  124.1  123.6  123.   122.3  122.   121.5  121.
  120.6  119.9  119.4  119.2  118.7  118.4  117.9  117.   116.9  116.1
  115.8  115.4  115.2  115.   

### Create $\sum$ matrix which is a diagonal matrix 

In [196]:
SIGMA = np.zeros((9724,1000))
for i in range(len(Sigma)):
    SIGMA[i][i] = Sigma[i]


### Truncated process
1. Define k
2. k is the number of value we want to keep in $\sum$ matrix, k < min(m,n) where m,n is the size of matrix A
3. Keep first k rows of $V^T$, first k columns of $U$, first k columns and rows of $\sum$ 
4. The new matrices are $U_k$, $V^T_k$ and $\sum_k$
5. Multiply three of them we get the new approximation matrix $A_k$

In [197]:

k = 500
SIGMA = SIGMA[0:k]
SIGMA = SIGMA[:,:k]
U = U[:, :k]
VT = VT[0:k]
print(U)
print(VT)
print(SIGMA)

[[-4.81284555e-02  9.09425495e-02  1.36862583e-02 ...  1.50412718e-02
  -1.44654335e-02  2.81760871e-02]
 [-2.06721903e-02  4.35999190e-02  1.76379408e-02 ...  5.24902051e-03
  -1.57936382e-02 -6.40659991e-02]
 [-4.35351901e-02  6.66115624e-02  2.13426028e-02 ... -5.12444237e-04
   3.79453044e-02 -5.95459743e-05]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
[[-0.0016     -0.02759527 -0.01049043 ... -0.00623383 -0.04717257
  -0.00287675]
 [ 0.00380467  0.03188476  0.0267767  ...  0.01508924  0.07970771
   0.00830164]
 [-0.00311137  0.01942773 -0.01625393 ...  0.00460073  0.09445035
   0.01208184]
 ...
 [-0.00248771 -0.03153242  0.05347393 ...  0.00607483 -0.01076908
  -0.00956524]
 [ 0.00469535  0.02280593 -0.006328

In [198]:
print(U.dot(SIGMA).dot(VT))

[[ 4.09980499 10.46158461 -0.03290023 ...  0.54885003 -0.13139795
   0.1471515 ]
 [ 0.41801659  7.63557857 -0.41653287 ... -0.17141302 -0.04890281
   0.07051624]
 [ 0.16292809  9.7833664   0.70695856 ...  0.19983553 -0.06369996
  -0.05372013]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


### New V = $SIGMA$ $V^T$

Then the matrix A can be represent by $A$ = $U$ $V$
- U represent the latent factor of users
- V represent the latent factor of items (after multiply with Sigma matrix)

Singular Value Decomposition (SVD) has an important property that makes it particularly interesting for our application. SVD provides the best low-rank linear approximation of the original matrix $A$. It is possible to retain only $k$ singular values by discarding others, resulting in a reduced matrix $S_k$. Since the singular values in $S$ are sorted such that $s_1 \geq s_2 \geq \ldots \geq s_r$, the reduction process involves retaining the first $k$ singular values. Matrices $U$ and $V$ are also reduced to produce matrices $U_k$ and $V_k$, respectively. $U_k$ is obtained by removing $(r - k)$ columns from matrix $U$, while $V_k$ is obtained by removing $(r - k)$ rows from matrix $V$. When these three reduced matrices are multiplied, we obtain a matrix $A_k$ given by $A_k = U_k S_k V_k^T$. This $A_k$ is a rank-$k$ matrix that provides the closest approximation to the original matrix $A$.



## Prediction generation using SVD. 
Once the $m \times n$ ratings matrix $R$ is decomposed and reduced into three SVD component matrices with $k$ features $U_k$, $S_k$, and $V_k$, prediction can be generated from it by computing the cosine similarities (dot products) between m pseudo-customers $U_k \cdot \sqrt{S_k}^T$ and $n$ pseudo-products $\sqrt{S_k} \cdot V_k^T$ . In particular, the prediction score $P_{i,j}$ for the $i$-th customer on the $j$-th product is obtained by adding the row average $r_i$ to the similarity. Formally, $P_{i,j} = r_i + U_k \cdot \sqrt{S_k}^T(i) \cdot \sqrt{S_k} \cdot V_k^T(j)$. Once the SVD decomposition is done, the prediction generation process involves only a dot product computation, which takes $O(1)$ time, since $k$ is a constant.


## Prediction formula of normalized utility matrix $A$

$$ \hat{r}_{ij} = \sum_{k=1}^{k} u_{ik} \cdot v_{jk} \cdot \sigma_k $$

Where:
- $\hat{r}_{ij}$  is the predicted rating of user \( i \) to item \( j \).
-  $u_{ik}$  is the  $(i,k)$ - th element of matrix $U$.
- $v_{jk}$  is the $(j,k)$ - th element of matrix $V^T$.
- $\sigma_k$ is the $k$-th singular value.






In [199]:
Wk = SIGMA.dot(VT)
predict_rating = []
for id in idx:
    score =  average + U[id].dot(Wk[:,1])
    predict_rating.append(score)
print("Predict rating is: ")
print(predict_rating)
print("True rating is :")
print(true_rating)

Predict rating is: 
[8.383415723039265, 7.804107333071544, 9.469016570455121, 9.103046736668501, 8.494919207291026, 7.880903744028679, 8.585581487926575, 8.303439901967735, 8.641613655052998, 9.061373180411067, 8.283688262149958, 9.004087452370635, 8.583192992553805, 8.87777739929206, 8.63821681771376, 8.736905404091859, 8.333229268947816, 8.031034967297005, 8.641993870699658, 8.659477811697474, 8.512199168646283, 8.924575684437327, 8.94114565999881, 8.245121029546798, 8.788088076163032, 7.822387591578393, 8.214018237346124, 8.524874154110371, 8.697979037447995, 8.369113546191368, 8.592989582960657, 8.674332338856713, 8.947612214408021, 8.651409317673158, 8.43, 9.169803645307557, 8.487326141759405, 8.118292427307981, 8.817378490948549, 8.088605175764659, 8.56170652367207, 8.557352937359655, 8.21270337411615]
True rating is :
[7.0, 10.0, 7.0, 7.0, 8.43, 9.0, 8.0, 10.0, 9.0, 10.0, 9.0, 10.0, 9.0, 9.0, 7.0, 9.0, 8.0, 7.0, 5.0, 5.0, 8.43, 8.0, 8.0, 7.0, 10.0, 7.0, 10.0, 8.0, 8.0, 9.0, 10.0

In [200]:
print(U[0].dot(Wk[:,1]))
print(A[0][1])

10.46158460632662
10.0


In [201]:
v1 = np.array(predict_rating)
v2 = np.array(true_rating)
print(np.mean((v1-v2)**2))

2.6385972781541107


In [202]:
print(np.mean(v2**2))

66.26336046511628
